In [ ]:
import pandas as pd
import numpy as np
from funciones import *

# Configuracion de display de DataFrames en Jupyter
pd.options.display.max_columns = None

# Localizacion del .CSV del dataset
data_location_raw = './datasets/properatti.csv'

# Localizacion del .CSV donde enviaremos los datos limpios
data_location_clean = './datasets/properatti_clean.csv'

# Hacemos la primera importacion de datos en un dataframe de raw_data
raw_data = pd.read_csv(data_location_clean, index_col=0)

# Copiamos el dataframe a otro para tener una copia literal del mismo
data = raw_data.copy()

# Forma de resetear el index del dataframe si no se usa el index_col=0
## data_default.reset_index(drop=True, inplace=True)
## data_str.index

Información del DataFrame RAW que hemos importado

In [ ]:
raw_data.info()

Información del DataFrame que vamos a utilizar para nuestro análisis

In [ ]:
data.info()

#### Máscaras para valores no nulos

In [ ]:
mask_price_notnull = data['price'].notnull()
mask_currency_notnull = data['currency'].notnull()
mask_price_aprox_local_currency_notnull = data['price_aprox_local_currency'].notnull()
mask_price_aprox_usd_notnull = data['price_aprox_usd'].notnull()
mask_surface_total_in_m2_notnull = data['surface_total_in_m2'].notnull()
mask_surface_covered_in_m2_notnull = data['surface_covered_in_m2'].notnull()
mask_price_usd_per_m2_notnull = data['price_usd_per_m2'].notnull()
mask_price_per_m2_notnull = data['price_per_m2'].notnull()

#### Máscaras para valores nulos

In [ ]:
mask_price_isnull = data['price'].isnull()
mask_currency_isnull = data['currency'].isnull()
mask_price_aprox_local_currency_isnull = data['price_aprox_local_currency'].isnull()
mask_price_aprox_usd_isnull = data['price_aprox_usd'].isnull()
mask_surface_total_in_m2_isnull = data['surface_total_in_m2'].isnull()
mask_surface_covered_in_m2_isnull = data['surface_covered_in_m2'].isnull()
mask_price_usd_per_m2_isnull = data['price_usd_per_m2'].isnull()
mask_price_per_m2_isnull = data['price_per_m2'].isnull()

### Análisis de la cotización del dólar

Buscaremos una cotización de cambio a dolar para imputar en aquellos registros donde no tengamos una tasa definida a partir de los valores de la tabla.

El cálculo lo haremos a partir de las columnas de price y de price_aprox_in_usd

In [ ]:
#mask = mask_price_notnull & mask_price_aprox_usd_notnull
#data_price_notnull = data[mask]

cotizacion_USD_ARS_raw = data['price'] / data['price_aprox_usd']
cotizacion_USD_ARS = cotizacion_USD_ARS_raw.dropna()

print(f"Cotizaciones de dolar: {len(cotizacion_USD_ARS.unique())}")
print(f"Distribución de las cotizaciones del dolar:\n{cotizacion_USD_ARS.value_counts()}")
#df_data = data
#df_data['cotizacion'] = cotizacion_dolar
#df_data['cotizacion'].isna().sum()

In [ ]:
import seaborn as sns

#cotizacion_dolar_cat = cotizacion_dolar.unique()
#len(cotizacion_dolar_cat)
#cotizacion_dolar_categorica = cotizacion_dolar.groupby(by=cotizacion_dolar_cat, axis=1)
#cotizacion_dolar_categorica
sns.displot(cotizacion_USD_ARS, kind="kde")

Observamos que hay una cotización en torno a 1, y otra a 17.5 por dolar, por lo cual intentaremos busacar las cotizaciones que se encuentren en un rago de +-0.5 de distancia.

In [ ]:
mask_cotizacion_notnull = cotizacion_USD_ARS.notnull()

mask_cotizacion_rango_0_1 = (cotizacion_USD_ARS >= 0.0) & (cotizacion_USD_ARS <= 1.0)
mask_cotizacion_rango_17_18 = (cotizacion_USD_ARS >= 17.0) & (cotizacion_USD_ARS <= 18.0)

mask_cotizacion_otras = mask_cotizacion_notnull & (~(mask_cotizacion_rango_0_1|mask_cotizacion_rango_17_18))



print(f"Observaciones con cotizaciones no nulas: {mask_cotizacion_notnull.sum()}.\n"
      f"Observaciones con cotizaciones rango 0 a 1: {mask_cotizacion_rango_0_1.sum()}.\n"
      f"Observaciones con cotizaciones rango 17 a 18: {mask_cotizacion_rango_17_18.sum()}.\n"
      f"Observaciones con otra cotizaciones: {mask_cotizacion_otras.sum()}.\n"
      f"Checksum cotizaciones: {mask_cotizacion_rango_0_1.sum()+mask_cotizacion_rango_17_18.sum()+mask_cotizacion_otras.sum()}.\n")

In [ ]:
# Descripción del dolar en cotizaciones en rango 0 a 1
print(f"Descripción cotizaciones en rago 0 a 1\n"
      f"{cotizacion_USD_ARS[mask_cotizacion_rango_0_1].describe()}")

In [ ]:
# Descripción del dolar en cotizaciones en rango 17 a 18
print(f"Descripción cotizaciones en rago 17 a 18\n"
      f"{cotizacion_USD_ARS[mask_cotizacion_rango_17_18].describe()}")

In [ ]:
# Descripción del dolar en otras cotizaciones
print(f"Descripción otras cotizaciones\n"
      f"{cotizacion_USD_ARS[mask_cotizacion_otras].describe()}")

In [ ]:
print(cotizacion_USD_ARS[mask_cotizacion_otras])

Defino las cotizaciones de cambio que usaremos para posibles imputaciones en registros donde esté el faltante.

In [ ]:
exchage_rate_USD_ARS = cotizacion_USD_ARS[mask_cotizacion_rango_17_18].mean()

exchage_rate_USD_PEN = cotizacion_USD_ARS[mask_cotizacion_otras & (data['currency']=='PEN')].mean()

exchage_rate_USD_UYU = cotizacion_USD_ARS[mask_cotizacion_otras & (data['currency']=='UYU')].mean()


Calculo del DataFram en el otro dentido de la cotización

In [ ]:
cotizacion_ARS_USD_raw = data['price_aprox_usd'] / data['price']
cotizacion_ARS_USD_raw

#### Hipótesis 1

Aquí ponemos como hipótesis que existe una realación lineal entre las variables price, surface_covered_in_m2, price_per_m2:

$$ \text{price} = \text{surface_covered_in_m2} \cdot \text{price_per_m2}$$

Y supondremos que para aquellos valores en donde esto se cumpla, los valores de las 3 variables son correctas y pueden usarse para imputaciones posteriores.
- price
- surface_covered_in_m2
- price_per_m2

In [ ]:
## Hipotesis 1

# La mascara de no nulos no es necesaria, sin embargo la dejamos para agregar claridad.
mask_hipotesis_uno_not_null = mask_price_notnull &  mask_surface_covered_in_m2_notnull & mask_price_per_m2_notnull

mask_hipotesis_uno_condicion_limite = abs(data['price'] - data['surface_covered_in_m2'] * data['price_per_m2']) < 1

mask_hipotesis_uno = mask_hipotesis_uno_not_null & mask_hipotesis_uno_condicion_limite

#### Hipótesis 2

Aquí ponemos como hipótesis que existe una realación lineal entre las variables price, surface_total_in_m2, price_per_m2:

$$ \text{price_aprox_usd} = \text{surface_total_in_m2} \cdot \text{price_usd_per_m2}$$

Y supondremos que para aquellos valores en donde esto se cumpla, los valores de las 3 variables son correctas y pueden usarse para imputaciones posteriores.
- price_aprox_usd
- surface_total_in_m2
- price_usd_per_m2

In [ ]:
## Hipotesis 2

# La mascara de no nulos no es necesaria, sin embargo la dejamos para agregar claridad.
mask_hipotesis_dos_not_null = mask_price_aprox_usd_notnull & mask_surface_total_in_m2_notnull & mask_price_usd_per_m2_notnull

mask_hipotesis_dos_condicion_limite = abs(data['price_aprox_usd'] - data['surface_total_in_m2'] * data['price_usd_per_m2']) < 1

mask_hipotesis_dos = mask_hipotesis_dos_not_null & mask_hipotesis_dos_condicion_limite

#### Imputación para propiedades en dólares
Para aquellas propiedades que están en dólares, extraeremos los valores que cumplan con alguna de las hipótesis que planteamos anteriormente.

In [ ]:
mask_currency_USD = data['currency'] == 'USD'

mask_prices_usd_notnull = mask_price_notnull & mask_price_aprox_usd_notnull

#Nos quedaremos unicamente con aquellas observaciones en donde ambos precios son iguales
mask_prices_usd_equal = data['price'] == data['price_aprox_usd']

##### Imputación en dolares: Hipótesis 1
Para el subconjuto de propiedades que describimos antes, evaluaremos el cumplimiento de la hipótesis 1, y para aquellos valores en donde se cumpla, imputaremos las siguientes variables:

- price_clean
- currency_clean
- surface_covered_in_m2_clean
- price_per_m2_covered_clean

In [ ]:
mask_imp_prop_usd_hipotesis_uno = mask_currency_USD \
                                & mask_prices_usd_notnull \
                                & mask_prices_usd_equal \
                                & mask_hipotesis_uno

print(f"Cantidad de propiedades en donde realizaremos imputaciones: {mask_imp_prop_usd_hipotesis_uno.sum()}")

Realizamos ahora la imputación en el dataset

In [ ]:
data.loc[mask_imp_prop_usd_hipotesis_uno, 'price_clean'] = data.loc[mask_imp_prop_usd_hipotesis_uno,'price']
data.loc[mask_imp_prop_usd_hipotesis_uno, 'currency_clean'] = data.loc[mask_imp_prop_usd_hipotesis_uno,'currency']
data.loc[mask_imp_prop_usd_hipotesis_uno, 'surface_covered_in_m2_clean'] = data.loc[mask_imp_prop_usd_hipotesis_uno,'surface_covered_in_m2']
data.loc[mask_imp_prop_usd_hipotesis_uno, 'price_per_m2_covered_clean'] = data.loc[mask_imp_prop_usd_hipotesis_uno,'price_per_m2']

----------------------------------------------------------

#### Imputación para propiedades en pesos ARS
Para aquellas propiedades que están en pesos argentinos, extraeremos los valores que cumplan con alguna de las hipótesis que planteamos anteriormente.

In [ ]:
mask_currency_ARS = data['currency'] == 'ARS'

mask_prices_ars_notnull = mask_price_notnull & mask_price_aprox_local_currency_notnull

mask_prices_ars_valid = abs((data['price'] - data['price_aprox_local_currency']) / data['price']) < 0.03

##### Imputación en pesos: Hipótesis 1
Para el subconjuto de propiedades que describimos antes, evaluaremos el cumplimiento de la hipótesis 1, y para aquellos valores en donde se cumpla, imputaremos las siguientes variables:

- price_clean
- currency_clean
- surface_covered_in_m2_clean
- price_per_m2_covered_clean

In [ ]:
mask_imp_prop_ars_hipotesis_uno = mask_currency_ARS \
                                & mask_prices_ars_notnull \
                                & mask_prices_ars_valid \
                                & mask_hipotesis_uno

print(f"Cantidad de propiedades en donde realizaremos imputaciones: {mask_imp_prop_ars_hipotesis_uno.sum()}")

Realizamos ahora la imputación en el dataset

In [ ]:
data.loc[mask_imp_prop_ars_hipotesis_uno, 'price_clean'] = data.loc[mask_imp_prop_ars_hipotesis_uno,'price']
data.loc[mask_imp_prop_ars_hipotesis_uno, 'currency_clean'] = data.loc[mask_imp_prop_ars_hipotesis_uno,'currency']
data.loc[mask_imp_prop_ars_hipotesis_uno, 'surface_covered_in_m2_clean'] = data.loc[mask_imp_prop_ars_hipotesis_uno,'surface_covered_in_m2']
data.loc[mask_imp_prop_ars_hipotesis_uno, 'price_per_m2_covered_clean'] = data.loc[mask_imp_prop_ars_hipotesis_uno,'price_per_m2']

#### Conversión a moneda uniforme
Vamos a convertir las observaciones limpias de ARS a USD utilizando la tasa de conversión que sale como consecuencia de la realación lineal que hay entre sus columnas; de este modo poremos realizar comparaciones de manera transparente sin requerir de calculos extras.

In [ ]:
mask_currency_ARS_data_clean = data['currency_clean'] == 'ARS'

data.loc[mask_currency_ARS_data_clean, 'price_clean'] = data.loc[mask_currency_ARS_data_clean,'price'] / cotizacion_USD_ARS_raw.loc[mask_currency_ARS_data_clean]
data.loc[mask_currency_ARS_data_clean, 'price_per_m2_covered_clean'] = data.loc[mask_currency_ARS_data_clean,'price_per_m2'] / cotizacion_USD_ARS_raw.loc[mask_currency_ARS_data_clean]

data.loc[mask_currency_ARS_data_clean, 'currency_clean'] = 'USD'

### Exportación de datos limpios

In [ ]:
data.to_csv(data_location_clean)